In [1]:
import os
import re
import torch
import random
import numpy as np
import pandas as pd

from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler
from konlpy.tag import Mecab, Okt

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# 1.데이터 처리 과정 살펴보기

In [3]:
data_dir = './data/ChatbotData.csv'
df = pd.read_csv(data_dir) ## Question, Answer, Label(emotion - 0 : normal, 1 : negative, 2 : positive)
print(df)

                             Q                         A  label
0                       12시 땡!                하루가 또 가네요.      0
1                  1지망 학교 떨어졌어                 위로해 드립니다.      0
2                 3박4일 놀러가고 싶다               여행은 언제나 좋죠.      0
3              3박4일 정도 놀러가고 싶다               여행은 언제나 좋죠.      0
4                      PPL 심하네                눈살이 찌푸려지죠.      0
...                        ...                       ...    ...
11818           훔쳐보는 것도 눈치 보임.        티가 나니까 눈치가 보이는 거죠!      2
11819           훔쳐보는 것도 눈치 보임.             훔쳐보는 거 티나나봐요.      2
11820              흑기사 해주는 짝남.                    설렜겠어요.      2
11821  힘든 연애 좋은 연애라는게 무슨 차이일까?  잘 헤어질 수 있는 사이 여부인 거 같아요.      2
11822               힘들어서 결혼할까봐        도피성 결혼은 하지 않길 바라요.      2

[11823 rows x 3 columns]


In [4]:
question = df['Q']
answer = df['A']

print(len(question), len(answer))

11823 11823


In [5]:
## 이 문자들을 제외한 나머지들은 모두 제거한다.
korean_pattern = r'[^ ?,.!A-Za-z0-9가-힣+]'
normalizer = re.compile(korean_pattern)
print(normalizer)

re.compile('[^ ?,.!A-Za-z0-9가-힣+]')


In [6]:
def normalize(sentence):
    return normalizer.sub("", sentence)

In [7]:
## 한글 형태소 분석
mecab = Mecab()
okt = Okt()

rand_idx = random.randint(0, len(question) - 1)
print(f"{question[rand_idx]} ---> {mecab.morphs(normalize(question[rand_idx]))}")

일이랑 공부 병행 가능? ---> ['일', '이랑', '공부', '병행', '가능', '?']


In [8]:
def clean_text(sentence, tagger):
    sentence = normalize(sentence)
    sentence = tagger.morphs(sentence)
    sentence = ' '.join(sentence)
    sentence = sentence.lower()

    return sentence

print(clean_text(question[rand_idx], okt))
print(clean_text(answer[rand_idx], okt))

일이 랑 공부 병행 가능 ?
시간 활용 에 따라 다르겠죠 .


In [9]:
questions = [clean_text(sent, okt) for sent in question[:1000]]
answers = [clean_text(sent, okt) for sent in answer[:1000]]

print(questions[:5])
print(answers[:5])

['12시 땡 !', '1 지망 학교 떨어졌어', '3 박 4일 놀러 가고 싶다', '3 박 4일 정도 놀러 가고 싶다', 'ppl 심하네']
['하루 가 또 가네요 .', '위로 해 드립니다 .', '여행 은 언제나 좋죠 .', '여행 은 언제나 좋죠 .', '눈살 이 찌푸려지죠 .']


In [10]:
## 단어 사전 생성
PAD_TOKEN = 0
SOS_TOKEN = 1
EOS_TOKEN = 2

class WordVocab():
    def __init__(self):
        self.word2count = {}
        
        self.word2index = {'<PAD>' : PAD_TOKEN,
                           '<SOS>' : SOS_TOKEN,
                           '<EOS>' : EOS_TOKEN}
        
        self.index2word = {PAD_TOKEN : '<PAD>',
                           SOS_TOKEN : '<SOS>',
                           EOS_TOKEN : '<EOS'}
        self.n_words = 3

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [11]:
rand_idx = random.randint(0, len(questions) - 1)
print(f"Original : {questions[rand_idx]}")

lang = WordVocab()
lang.add_sentence(questions[rand_idx])
print(lang.word2index)

Original : 내 문제 는 뭘 까
{'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '내': 3, '문제': 4, '는': 5, '뭘': 6, '까': 7}


In [12]:
## batch를 위해 문장의 길이를 맞춰준다. padding
max_length = 10
sentence_length = 6

sentence_tokens = np.random.randint(low=3, high=100, size=(sentence_length,))
sentence_tokens = sentence_tokens.tolist()
print(sentence_tokens)

[27, 37, 64, 65, 57, 14]


In [13]:
sentence_tokens = sentence_tokens[ : (max_length - 1)]
token_length = len(sentence_tokens)

print(sentence_tokens)
print(token_length)

[27, 37, 64, 65, 57, 14]
6


In [14]:
sentence_tokens.append(2) ## <EOS> token 추가.
for i in range(token_length, max_length - 1):
    sentence_tokens.append(PAD_TOKEN)

print(f"output : {sentence_tokens}")
print(f"total length: {len(sentence_tokens)}")

output : [27, 37, 64, 65, 57, 14, 2, 0, 0, 0]
total length: 10


# 2.Train, Test Dataset

In [15]:
class TextDataset(Dataset):
    def __init__(self, csv_path, min_length=3, max_length=32):
        super(TextDataset, self).__init__()

        ## Token 정의.
        self.PAD_TOKEN = 0
        self.SOS_TOKEN = 1
        self.EOS_TOKEN = 2

        self.tagger = Mecab() ## 형태소 분석기
        self.max_length = max_length ## 한 문장의 최대 길이

        df = pd.read_csv(csv_path)
        korean_pattern = r'[^ ?,.!A-Za-z0-9가-힣+]'
        self.normalizer = re.compile(korean_pattern)

        self.source_clean = []
        self.target_clean = []
        self.wordvocab = WordVocab()
        for _, row in df.iterrows():
            source = row['Q'] ## raw questions
            target = row['A'] ## raw answers

            source = self.clean_text(source)
            target = self.clean_text(target)

            if len(source.split()) > min_length and len(target.split()) > min_length:
                self.wordvocab.add_sentence(source)
                self.wordvocab.add_sentence(target)
                self.source_clean.append(source)
                self.target_clean.append(target)


    def __getitem__(self, idx):
        inputs = self.source_clean[idx] ## n번째 (형태소로 분리된) 문장 하나를 가져온다. ex) 내 의견 을 존중 해줬으면
        inputs_sentences = self.texts_to_sequences(inputs) ## ex) {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '내': 3, '의견': 4, '을': 5, '존중': 6, '해줬으면': 7}
        inputs_padded = self.pad_sequence(inputs_sentences)

        outputs = self.target_clean[idx]
        outputs_sequences = self.texts_to_sequences(outputs)
        outputs_padded = self.pad_sequence(outputs_sequences)

        return torch.tensor(inputs_padded), torch.tensor(outputs_padded)
    
    
    def __len__(self):
        return len(self.source_clean)


    def normalize(self, sentence):
        return self.normalizer.sub("", sentence)


    def clean_text(self, sentence):
        sentence = self.normalize(sentence)
        sentence = self.tagger.morphs(sentence) ## 문장을 단어 단위로 분리한다.(형태소 분석)
        sentence = ' '.join(sentence)
        sentence = sentence.lower()

        return sentence


    def texts_to_sequences(self, sentence):
        return [self.wordvocab.word2index[w] for w in sentence.split()] ## 각 형태소를 key로, index를 부여.


    def pad_sequence(self, sentence_tokens):
        sentence_tokens = sentence_tokens[ : (self.max_length - 1)]
        token_length = len(sentence_tokens)
        
        sentence_tokens.append(self.EOS_TOKEN)
        for i in range(token_length, (self.max_length - 1)):
            sentence_tokens.append(self.PAD_TOKEN)

        return sentence_tokens    

In [16]:
MAX_LENGTH = 50
dataset = TextDataset("./data/ChatbotData.csv", min_length=3, max_length=MAX_LENGTH)
x, y = dataset[10]
print(f"Question Tensor : {x.shape} \n {x}")
print(f"Answer Tensor : {y.shape} \n {y}")

Question Tensor : torch.Size([50]) 
 tensor([83, 84, 51, 85, 86, 18,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
Answer Tensor : torch.Size([50]) 
 tensor([87, 88, 58, 89, 63, 90, 11,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])


In [17]:
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
print(train_size, test_size)

8168 2042


In [18]:
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset)

x, y = next(iter(train_dataloader))
print(x.shape, y.shape)

torch.Size([16, 50]) torch.Size([16, 50])


# 3.Seq2Seq model

## 3-1.embedding layer

In [19]:
embedding_dim = 64
embedding = nn.Embedding(num_embeddings=dataset.wordvocab.n_words, embedding_dim=embedding_dim) ## embedding할 단어의 수, embedding 차원

embedded = embedding(x)
print(f"{x.shape} --> {embedded.shape}")

# embedded = embedded.permute(1, 0, 2)
# print(embedded.shape)

torch.Size([16, 50]) --> torch.Size([16, 50, 64])


## 3-2.LSTM

In [20]:
class Encoder(nn.Module):
    def __init__(self, num_vocabs, hidden_size, embedding_dim, num_layers):
        super(Encoder, self).__init__()
        
        # 단어 사전의 개수 지정
        self.num_vocabs = num_vocabs
        # 임베딩 레이어 정의 (number of vocabs, embedding dimension)
        self.embedding = nn.Embedding(num_vocabs, embedding_dim)
        # GRU (embedding dimension)
        self.gru = nn.GRU(embedding_dim, 
                          hidden_size, 
                          num_layers=num_layers, 
                          bidirectional=False)
        
    def forward(self, x):
        x = self.embedding(x).permute(1, 0, 2)
        output, hidden = self.gru(x)
        return output, hidden


In [21]:
class Decoder(nn.Module):
    def __init__(self, num_vocabs, hidden_size, embedding_dim, num_layers=1, dropout=0.2):
        super(Decoder, self).__init__()
        # 단어사전 개수
        self.num_vocabs = num_vocabs
        self.embedding = nn.Embedding(num_vocabs, embedding_dim)
        self.dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(embedding_dim, 
                          hidden_size, 
                          num_layers=num_layers, 
                          bidirectional=False)
        
        # 최종 출력은 단어사전의 개수
        self.fc = nn.Linear(hidden_size, num_vocabs)
        
    def forward(self, x, hidden_state):
        x = x.unsqueeze(0) # (1, batch_size) 로 변환
        embedded = F.relu(self.embedding(x))
        embedded = self.dropout(embedded)
        output, hidden = self.gru(embedded, hidden_state)
        output = self.fc(output.squeeze(0)) # (sequence_length, batch_size, hidden_size(32) x bidirectional(1))
        return output, hidden

In [22]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, inputs, outputs, teacher_forcing_ratio=0.5):
        batch_size, output_length = outputs.shape
        output_num_vocabs = self.decoder.num_vocabs

        predicted_outputs = torch.zeros(output_length, batch_size, output_num_vocabs).to(self.device)
        _, decoder_hidden = self.encoder(inputs)
        decoder_input = torch.full((batch_size, ), SOS_TOKEN, device=self.device)

        for t in range(0, output_length):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            predicted_outputs[t] = decoder_output
            
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = decoder_output.argmax(1)
            decoder_input = outputs[:, t] if teacher_force else top1

        return predicted_outputs.permute(1, 0, 2)

# 4.Training

In [23]:
NUM_VOCABS = dataset.wordvocab.n_words
HIDDEN_DIM = 512
EMBEDDING_DIM = 256

# Encoder 정의
encoder = Encoder(num_vocabs=dataset.wordvocab.n_words, 
                       hidden_size=32, 
                       embedding_dim=64, 
                       num_layers=1)
# Decoder 정의
decoder = Decoder(num_vocabs=dataset.wordvocab.n_words, 
                       hidden_size=32, 
                       embedding_dim=64, 
                       num_layers=1)

model = Seq2Seq(encoder.to(device), decoder.to(device), device)
print(model)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(6417, 64)
    (gru): GRU(64, 32)
  )
  (decoder): Decoder(
    (embedding): Embedding(6417, 64)
    (dropout): Dropout(p=0.2, inplace=False)
    (gru): GRU(64, 32)
    (fc): Linear(in_features=32, out_features=6417, bias=True)
  )
)


In [24]:
class EarlyStopping:
    def __init__(self, patience=3, delta=0.0, mode='min', verbose=True):
        """
        patience (int): loss or score가 개선된 후 기다리는 기간. default: 3
        delta  (float): 개선시 인정되는 최소 변화 수치. default: 0.0
        mode     (str): 개선시 최소/최대값 기준 선정('min' or 'max'). default: 'min'.
        verbose (bool): 메시지 출력. default: True
        """
        self.early_stop = False
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        
        self.best_score = np.Inf if mode == 'min' else 0
        self.mode = mode
        self.delta = delta
        

    def __call__(self, score):

        if self.best_score is None:
            self.best_score = score
            self.counter = 0
        elif self.mode == 'min':
            if score < (self.best_score - self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')
                
        elif self.mode == 'max':
            if score > (self.best_score + self.delta):
                self.counter = 0
                self.best_score = score
                if self.verbose:
                    print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
            else:
                self.counter += 1
                if self.verbose:
                    print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                          f'Best: {self.best_score:.5f}' \
                          f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')
                
            
        if self.counter >= self.patience:
            if self.verbose:
                print(f'[EarlyStop Triggered] Best Score: {self.best_score:.5f}')
            # Early Stop
            self.early_stop = True
        else:
            # Continue
            self.early_stop = False

In [25]:
LR = 1e-3
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

es = EarlyStopping(patience=10, 
                   delta=0.001, 
                   mode='min', 
                   verbose=True)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 mode='min', 
                                                 factor=0.5, 
                                                 patience=2,
                                                 threshold_mode='abs',
                                                 min_lr=1e-8, 
                                                 verbose=True)

In [26]:
def train(model, dataloader, optimizer, loss_func, device):
    model.train()
    
    avg_loss = 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        output = model(x, y)
        output_dim = output.size(2)

        output = output.reshape(-1, output_dim)
        y = y.view(-1)

        loss = loss_func(output, y)
        loss.backward()
        optimizer.step()

        avg_loss += loss.item() * x.size(0)

    return avg_loss / len(dataloader)


def evaluate(model, data_loader, loss_fn, device):
    model.eval()
    
    eval_loss = 0
    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            output = model(x, y)
            output_dim = output.size(2)
            output = output.reshape(-1, output_dim)
            y = y.view(-1)
            
            # Loss 계산
            loss = loss_fn(output, y)
            
            eval_loss += loss.item() * x.size(0)
            
    return eval_loss / len(data_loader)

In [27]:
def sequence_to_sentence(sequences, index2word):
    outputs = []
    for p in sequences:

        word = index2word[p]
        if p not in [SOS_TOKEN, EOS_TOKEN, PAD_TOKEN]:
            outputs.append(word)
        if word == EOS_TOKEN:
            break
    return ' '.join(outputs)

def random_evaluation(model, dataset, index2word, device, n=10):
    
    n_samples = len(dataset)
    indices = list(range(n_samples))
    np.random.shuffle(indices)      # Shuffle
    sampled_indices = indices[:n]   # Sampling N indices
    
    # 샘플링한 데이터를 기반으로 DataLoader 생성
    sampler = SubsetRandomSampler(sampled_indices)
    sampled_dataloader = DataLoader(dataset, batch_size=10, sampler=sampler)
    
    model.eval()
    with torch.no_grad():
        for x, y in sampled_dataloader:
            x, y = x.to(device), y.to(device)        
            output = model(x, y, teacher_forcing_ratio=0)
            # output: (number of samples, sequence_length, num_vocabs)
            
            preds = output.detach().cpu().numpy()
            x = x.detach().cpu().numpy()
            y = y.detach().cpu().numpy()
            
            for i in range(n):
                print(f'질문   : {sequence_to_sentence(x[i], index2word)}')
                print(f'답변   : {sequence_to_sentence(y[i], index2word)}')
                print(f'예측답변: {sequence_to_sentence(preds[i].argmax(1), index2word)}')
                print('==='*10)

In [28]:
NUM_EPOCHS = 100
STATEDICT_PATH = '/home/pervinco/Models/seq2seq-chatbot-kor.pt'

best_loss = np.inf
for epoch in range(NUM_EPOCHS):
    loss = train(model, train_dataloader, optimizer, loss_func, device)
    
    val_loss = evaluate(model, test_dataloader, loss_func, device)
    
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), STATEDICT_PATH)
    
    if epoch % 5 == 0:
        print(f'epoch: {epoch+1}, loss: {loss:.4f}, val_loss: {val_loss:.4f}')
    
    # Early Stop
    es(loss)
    if es.early_stop:
        break
    
    # Scheduler
    scheduler.step(val_loss)
                   
model.load_state_dict(torch.load(STATEDICT_PATH))
torch.save(model.state_dict(), f'/home/pervinco/Models/seq2seq-chatbot-kor-{best_loss:.4f}.pt')

epoch: 1, loss: 30.6700, val_loss: 1.1377
[EarlyStopping] (Update) Best Score: 30.66996
[EarlyStopping] (Update) Best Score: 17.14138
[EarlyStopping] (Update) Best Score: 15.91111
[EarlyStopping] (Update) Best Score: 15.32331
[EarlyStopping] (Update) Best Score: 15.01190
epoch: 6, loss: 14.7307, val_loss: 0.9392
[EarlyStopping] (Update) Best Score: 14.73070
[EarlyStopping] (Update) Best Score: 14.57431
[EarlyStopping] (Update) Best Score: 14.47149
[EarlyStopping] (Update) Best Score: 14.34900
[EarlyStopping] (Update) Best Score: 14.16047
epoch: 11, loss: 14.0677, val_loss: 0.9068
[EarlyStopping] (Update) Best Score: 14.06774
[EarlyStopping] (Update) Best Score: 14.01093
[EarlyStopping] (Update) Best Score: 13.80560
[EarlyStopping] (Update) Best Score: 13.72863
[EarlyStopping] (Update) Best Score: 13.54630
epoch: 16, loss: 13.4767, val_loss: 0.8917
[EarlyStopping] (Update) Best Score: 13.47668
[EarlyStopping] (Update) Best Score: 13.38095
[EarlyStopping] (Update) Best Score: 13.28874
[E

In [29]:
model.load_state_dict(torch.load(STATEDICT_PATH))
random_evaluation(model, test_dataset, dataset.wordvocab.index2word, device)

질문   : 썸 타 다가 이제 안 볼 거 면 걍 잠수 타 ? 만나 서 말 해 ?
답변   : 애매 함 이 좋 은지 생각 해 보 세요 .
예측답변: 사랑 은 하 는 게 좋 겠 어요 .
질문   : 전 여친 카톡 메인 프로필 .
답변   : 카톡 보 지 마요 .
예측답변: 저 이 이 네요 .
질문   : 가슴 이 허락 하 질 않 네 .
답변   : 마음 이 하 라는 대로 해야죠 .
예측답변: 사랑 은 하 는 게 .
질문   : 결혼 준비 하 면서 못 보 던 모습 을 보 게 돼
답변   : 본 모습 일지 도 몰라요 .
예측답변: 사랑 은 하 는 게 좋 겠 어요 .
질문   : 여행 왔 는데 좋 아 하 는 선물 로 뭐 가 괜찮 을까 ?
답변   : 여행지 에서 만 살 수 있 는 걸로 골라 보 세요 .
예측답변: 사랑 은 하 는 게 좋 겠 어요 .
질문   : 취미 좀 만들 어 볼까 ?
답변   : 뭐 든 시작 해 보 면 좋 을 거 예요 .
예측답변: 사랑 은 하 는 게 좋 겠 어요 .
질문   : 마음 맞 는 사람 이 있 을까 ?
답변   : 서로 에게 맞춰 갈 수 있 을 거 예요 .
예측답변: 사랑 은 하 는 게 좋 겠 어요 .
질문   : 썸 을 오래 탔 는데 사귀 어도 되 는 거 야 ?
답변   : 최대한 빨리 사귀 는 게 좋 을 거 같 아요 .
예측답변: 사랑 은 하 는 게 좋 겠 어요 .
질문   : 그 시절 엔 다 그랬 지
답변   : 추억 에 잠길 때 도 필요 해요 .
예측답변: 저 이 이 네요 .
질문   : 이제 헤어진지 1 주일 됐 어
답변   : 긴 시간 이 었 을 거 라 생각 해요 .
예측답변: 좋 은 하 는 게 .
